In [ ]:
import popcp
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

dir(popcp)

# Simulations on Gaussian Data (§ 4.1-4.2)

## Effect of sample size on estimation

In [ ]:
M = 10000

In [ ]:
# Balanced samples
# Sample size ranges from 20 to 2000, step size = 20
# Gaussian distributions, identity covariance, equal displacement in all directions

sims_SS = {}
dims = [2, 8, 32, 128]
estimators = ['naive', 'halfnhalf', 'unregularized_pooled', 'regularized_pooled']

for estimator in estimators:
    for dim in dims:
        distance_scaling = (2/dim)**(1/2)
        displacement = 0.2*distance_scaling
        mu1 = np.zeros(dim)
        mu2 = mu1 + displacement
        cov1 = np.identity(dim)
        cov2 = np.identity(dim)
        
        sims_SS[(estimator, str(dim)+'D')] = []
        
        for sample_size in range(20,2000,20):
            N1 = sample_size//2
            N2 = sample_size//2

            true_CP = popcp.gaussian_ND_true_CP(mu1, mu2, cov1, cov2)
            estimate, var = popcp.monte_carlo_CP(mu1, mu2, cov1, cov2, estimator, N1, N2, M)
            sims_SS[(estimator, str(dim)+'D')].append([sample_size, estimate - true_CP, var])
        
        sims_SS[(estimator, str(dim)+'D')] = np.array(sims_SS[(estimator, str(dim)+'D')])

In [ ]:
os.chdir('/home/amyf')
with open('sims_SS.pickle', 'wb') as handle:
    pickle.dump(sims_SS, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Effect of true CP on estimation

In [ ]:
M = 10000
# distances in 2D that give equally spaced grid of true CPs
trueCPs_grid_2D = np.load('/home/amyf/trueCPs_grid_2D.npy')

In [ ]:
# Balanced samples
# True CP increases from 0.5 to 1, step size = 0.005
# Gaussian distributions, identity covariance, equal displacement in all directions

sims_tCP = {}
dims = [2, 8, 32, 128]
sample_sizes = [20, 200, 2000]
estimators = ['naive', 'halfnhalf', 'unregularized_pooled', 'regularized_pooled']

for estimator in estimators:
    for dim in dims:
        distance_scaling = (2/dim)**(1/2)
        mu1 = np.zeros(dim)
        cov1 = np.identity(dim)
        cov2 = np.identity(dim)

        for sample_size in sample_sizes:
            N1 = sample_size//2
            N2 = sample_size//2
            
            sims_tCP[(estimator, str(dim)+'D', sample_size)] = []
            
            for dist in trueCPs_grid_2D*distance_scaling:
                mu2 = mu1 + dist

                true_CP = popcp.gaussian_ND_true_CP(mu1, mu2, cov1, cov2)
                estimate, var = popcp.monte_carlo_CP(mu1, mu2, cov1, cov2, estimator, N1, N2, M)
                sims_tCP[(estimator, str(dim)+'D', sample_size)].append([true_CP, estimate - true_CP, var])
                
            sims_tCP[(estimator, str(dim)+'D', sample_size)] = np.array(sims_tCP[(estimator, str(dim)+'D', sample_size)])

In [ ]:
os.chdir('/home/amyf')
with open('sims_tCP.pickle', 'wb') as handle:
    pickle.dump(sims_tCP, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Synthetic Bias Correction (§ 4.4)

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
with open('/home/amyf/pickles/sims_tCP.pickle', 'rb') as handle:
    sims_tCP = pickle.load(handle)

In [ ]:
def all_points_delta_method(points, smoothness):
    """
    Performs delta method on all points given.
    
    Parameters:
        points (ndarray): column 1 - estimated CPs, column 2 - true CPs, 
                        column 3 - variance in estimated CP (each point is stored in one row)
        smoothness (scalar): smoothness of spline interpolation
        
    """
    
    points = points[points[:,0].argsort()]
    
    f = popcp.interpolate.splrep(points[:,0], points[:,1], k=5, s=smoothness)
    fprime = popcp.interpolate.splder(f)
    
    deriv = popcp.interpolate.splev(points[:,0], fprime)
    real_var = points[:,2] * (deriv**2) 
    return real_var

In [ ]:
dims = [2, 8, 32, 128]
sample_sizes = [20, 200, 2000]
true_CPs = np.array([0.5,0.6,0.7,0.8,0.9,1.0])
synthetic_bc = {}

for dim in dims:
    distance_scaling = (2/dim)**(1/2)
    
    for sample_size in sample_sizes:
        N1 = sample_size//2
        N2 = sample_size //2

        points = sims_tCP['regularized_pooled', str(dim)+'D', sample_size]
        # swap bias and true CP columns so bias is first
        points[:,[0,1]] = points[:,[1,0]]
        # recover estimated CP from bias
        points[:,0] = points[:,0] + points[:,1]
        SDs = np.sqrt(all_points_delta_method(points, 0.001))
        # delta method done for all points in grid, interpolate to get values for intermediate points
        f_upper = interp1d(points[:,0], points[:,1] + 2*SDs, fill_value='extrapolate')
        f_middle = interp1d(points[:,0], points[:,1], fill_value='extrapolate')
        f_lower = interp1d(points[:,0], points[:,1] - 2*SDs, fill_value='extrapolate')
        
        dists = np.array(trueCPs_grid_2D[[0,20,40,60,80,100]]) * distance_scaling
        
        synthetic_bc[(str(dim)+'D', sample_size)] = []

        for dist, true_CP in zip(dists, true_CPs):
            for i in range(5):
                #simulate to get an estimated CP
                x1 = np.random.multivariate_normal(np.zeros(dim), np.identity(dim), N1)
                x2 = np.random.multivariate_normal(np.zeros(dim) + dist, np.identity(dim), N2)
                e_CP = popcp.regularized_pooled_estimator(x1, x2)

                # get confidence interval
                CI_upper = f_upper(e_CP).tolist()
                CI_middle = f_middle(e_CP).tolist()
                CI_lower = f_lower(e_CP).tolist()
                synthetic_bc[(str(dim)+'D', sample_size)].append([true_CP, e_CP, CI_middle, CI_lower, CI_upper])
        
        synthetic_bc[(str(dim)+'D', sample_size)] = np.array(synthetic_bc[(str(dim)+'D', sample_size)])

In [ ]:
os.chdir('/home/amyf')
with open('synthetic_bc.pickle', 'wb') as handle:
    pickle.dump(synthetic_bc, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Real Data from Zhao et. al (§ 5.1-5.2)

## Frozen trials data

In [ ]:
frozen_data = pd.read_csv('/home/amyf/real data/Frozen data/frozen_trials.csv')
frozen_zeros = frozen_data.loc[frozen_data['choice'] == 1]
frozen_zeros = frozen_zeros.sample(frac=1).reset_index(drop=True)
frozen_ones = frozen_data.loc[frozen_data['choice'] == 2]
frozen_ones = frozen_ones.sample(frac=1).reset_index(drop=True)
frozen_data = frozen_zeros.append(frozen_ones, ignore_index=True)

print(frozen_data['choice'].value_counts())

In [ ]:
frozen_ind_CPs = []
for i in range(27):
    CP = popcp.choice_prob_1D(frozen_data.iloc[:, i], frozen_data['choice'])
    if (CP < 0.5):
        CP = 1 - CP
    frozen_ind_CPs.append([i + 1, CP])
    
frozen_ind_CPs.sort(key=lambda x:1-x[1])
frozen_ind_CPs = np.array(frozen_ind_CPs)

In [ ]:
def neuron_subset_CP(data, neurons, N1, N2):
    """
    Calculates CP for a subset of neurons.
    
    Paramters:
        data (dataframe): trials with neuron recordings
        neurons (list or ndarray): neurons to be used
        N1 (int): number of positive trials
        N2 (int): number of negative trials
    """
    
    data_used = np.array(data.iloc[:, [neuron - 1 for neuron in neurons]])
    positive_trials = data_used[:N1, :]
    negative_trials = data_used[-N2:, :]
    
    return popcp.estimate_CP(negative_trials, positive_trials, 'regularized_pooled')

In [ ]:
import random

neurons_2D_random = []
for i in range(5):
    neurons = random.sample(range(1, 28), 2)
    neurons_2D_random.append(neurons)
    
neurons_2D_best = []
for i in range(0, 10, 2):
    neurons_2D_best.append([int(n) for n in frozen_ind_CPs[i:i+2, 0].tolist()])

neurons_8D_random = []
for i in range(5):
    neurons = random.sample(range(1, 28), 8)
    neurons_8D_random.append(neurons)
    
neurons_8D_best = []
for i in range(5):
    neurons = random.sample(range(1, 14), 8)
    neurons_8D_best.append(neurons)

all_neurons = [np.arange(1,28)]

In [ ]:
all_neurons = [np.arange(1,28)]

In [ ]:
frozen_trials_analysis = {}
N1 = 43
N2 = 43

groups = [(2, '2D_random', neurons_2D_random), (2, '2D_best', neurons_2D_best), (8, '8D_random', neurons_8D_random), (8, '8D_best', neurons_8D_best), (27, 'all', all_neurons)]

for group in groups:
    dim = group[0]
    cov1 = np.identity(dim)
    cov2 = np.identity(dim)
    
    frozen_trials_analysis[(group[0],group[1])] = []
    
    for subset in group[2]:
        eCP = neuron_subset_CP(frozen_data, subset, N1, N2)
        CI = popcp.estimate_CI(eCP, 'regularized_pooled', N1, N2, dim, cov1, cov2, 2, 1)
        frozen_trials_analysis[(group[0],group[1])].append(CI)

In [ ]:
for subset in all_neurons:
    eCP = neuron_subset_CP(frozen_data, subset, N1, N2)
    CI = popcp.estimate_CI(eCP, 'regularized_pooled', N1, N2, dim, cov1, cov2, 2, 1)
    frozen_trials_analysis[27, 'all'].append(CI)

In [ ]:
for key, item in frozen_trials_analysis.items():
    frozen_trials_analysis[key] = np.asarray(item)

In [ ]:
os.chdir('/home/amyf')
with open('frozen_trials_analysis.pickle', 'wb') as handle:
    pickle.dump(frozen_trials_analysis, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Weak trials data

In [ ]:
weak_data = pd.read_csv('/home/amyf/real data/Weak data/weak_trials.csv')
weak_data.columns=['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14','N15','N16','N17','N18','N19','N20','N21','N22','N23','N24','N25','N26','N27','choice']
weak_zeros = weak_data.loc[weak_data['choice'] == 0]
weak_zeros = weak_zeros.sample(frac=1).reset_index(drop=True)
weak_ones = weak_data.loc[weak_data['choice'] == 1]
weak_ones = weak_ones.sample(frac=1).reset_index(drop=True)
weak_data = weak_zeros.append(weak_ones, ignore_index=True)

print(weak_data['choice'].value_counts())

In [ ]:
weak_ind_CPs = []
for i in range(27):
    CP = popcp.choice_prob_1D(weak_data.iloc[:, i], weak_data['choice'])
    if (CP < 0.5):
        CP = 1 - CP
    weak_ind_CPs.append([i + 1, CP])
    
weak_ind_CPs.sort(key=lambda x:1-x[1])
weak_ind_CPs = np.array(weak_ind_CPs)

In [ ]:
import random

neurons_2D_random = []
for i in range(5):
    neurons = random.sample(range(1, 28), 2)
    neurons_2D_random.append(neurons)
    
neurons_2D_best = []
for i in range(0, 10, 2):
    neurons_2D_best.append([int(n) for n in weak_ind_CPs[i:i+2, 0].tolist()])

neurons_8D_random = []
for i in range(5):
    neurons = random.sample(range(1, 28), 8)
    neurons_8D_random.append(neurons)
    
neurons_8D_best = []
for i in range(5):
    neurons = random.sample(range(1, 14), 8)
    neurons_8D_best.append(neurons)

all_neurons = [np.arange(1,28)]

In [ ]:
weak_trials_analysis = {}
N1 = 236
N2 = 236

groups = [(2, '2D_random', neurons_2D_random), (2, '2D_best', neurons_2D_best), (8, '8D_random', neurons_8D_random), (8, '8D_best', neurons_8D_best), (27, 'all', all_neurons)]

for group in groups:
    dim = group[0]
    cov1 = np.identity(dim)
    cov2 = np.identity(dim)
    
    weak_trials_analysis[(group[0],group[1])] = []
    
    for subset in group[2]:
        eCP = neuron_subset_CP(weak_data, subset, N1, N2)
    
        CI = popcp.estimate_CI(eCP, 'regularized_pooled', N1, N2, dim, cov1, cov2, 2, 1)
        weak_trials_analysis[(group[0],group[1])].append(CI)

In [ ]:
for key, item in weak_trials_analysis.items():
    weak_trials_analysis[key] = np.asarray(item)

In [ ]:
os.chdir('/home/amyf')
with open('weak_trials_analysis.pickle', 'wb') as handle:
    pickle.dump(weak_trials_analysis, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Subsampling weak trials data

In [ ]:
weak_data_copy = weak_data.copy()
weak_zeros_copy = weak_data_copy.loc[weak_data_copy['choice'] == 0]
weak_ones_copy = weak_data_copy.loc[weak_data_copy['choice'] == 1]

In [ ]:
weak_trials_sanity_check = {}
all_neurons = np.arange(1,28)
sample_sizes = [86, 200, 300, 400, 472]
dim = 27
cov1 = np.identity(dim)
cov2 = np.identity(dim)

for sample_size in sample_sizes:
    N1 = sample_size//2
    N2 = sample_size//2
    
    weak_trials_sanity_check[sample_size] = []
    
    for seed in range(25):
        weak_zeros_copy = weak_zeros_copy.sample(frac=1, random_state=seed).reset_index(drop=True)
        weak_ones_copy = weak_ones_copy.sample(frac=1, random_state=seed+25).reset_index(drop=True)
        weak_data_copy = weak_zeros_copy.append(weak_ones_copy, ignore_index=True)
        eCP = neuron_subset_CP(weak_data_copy, all_neurons, N1, N2)
        CI = popcp.estimate_CI(eCP, 'regularized_pooled', N1, N2, dim, cov1, cov2, 2, 1)
        
        weak_trials_sanity_check[sample_size].append(CI)

In [ ]:
for key, item in weak_trials_sanity_check.items():
    weak_trials_sanity_check[key] = np.asarray(item)

In [ ]:
os.chdir('/home/amyf')
with open('weak_trials_sanity_check.pickle', 'wb') as handle:
    pickle.dump(weak_trials_sanity_check, handle, protocol=pickle.HIGHEST_PROTOCOL)